## Retrieval of related information based on query from multiple database with access level rules logic and RAG answer

Creating Fake Database from different departments

In [22]:
import sqlite3
from datetime import datetime
import random

# Define departments with document data, limiting access levels to Employee, Executive, Manager, and Director
departments = {
    "Finance": [
        ("Financial Report Q1", "This is the content of the Finance Q1 report.", "Executive"),
        ("Investment Analysis", "Detailed investment strategies and outcomes.", "Manager"),
        ("Annual Budget", "Overview of the annual budget allocations and expenses.", "Executive"),
        ("Expense Tracking", "Monthly tracking of departmental expenses.", "Employee"),
        ("Risk Assessment", "Assessment of financial risks and mitigation strategies.", "Employee"),
        ("Quarterly Forecast", "Forecast of revenue and expenditure for the upcoming quarter.", "Executive"),
    ],
    "HR": [
        ("Policy Update", "New HR policies on leave and remote work.", "Employee"),
        ("Employee Satisfaction Report", "Analysis of employee satisfaction.", "Manager"),
        ("Onboarding Guide", "Steps and procedures for new employee onboarding.", "Employee"),
        ("Performance Review Process", "Guidelines for conducting employee performance reviews.", "Manager"),
        ("Diversity & Inclusion Initiatives", "Plans to enhance diversity and inclusion within the organization.", "Executive"),
        ("Health & Safety Policy", "Updated policies on health and workplace safety.", "Employee"),
    ],
    "Technical": [
        ("System Architecture", "Technical specifications for system architecture.", "Employee"),
        ("Tech Roadmap", "Roadmap for upcoming tech projects and improvements.", "Manager"),
        ("API Documentation", "Documentation for public and private APIs.", "Employee"),
        ("Database Optimization", "Strategies for optimizing database performance.", "Employee"),
        ("Cybersecurity Guidelines", "Security protocols for protecting data and infrastructure.", "Employee"),
        ("Release Notes", "Notes and changes for the latest software release.", "Manager"),
    ],
    "Marketing": [
        ("Marketing Strategy Q2", "Strategies and campaigns planned for Q2.", "Executive"),
        ("Customer Feedback Analysis", "Analysis of customer feedback on recent campaigns.", "Employee"),
        ("Brand Guidelines", "Updated guidelines for branding and marketing materials.", "Employee"),
        ("Social Media Content Plan", "Plan for social media posts and engagements.", "Employee"),
        ("Market Research Report", "Report on market trends and customer behavior.", "Employee"),
        ("Campaign Performance", "Performance metrics for recent marketing campaigns.", "Executive"),
    ],
    "Sales": [
        ("Sales Forecast", "Projected sales figures for the next quarter.", "Executive"),
        ("Client Onboarding Guide", "Steps to onboard new clients effectively.", "Employee"),
        ("Lead Scoring Model", "Model for scoring and prioritizing sales leads.", "Manager"),
        ("Product Catalog", "Comprehensive catalog of products with descriptions.", "Employee"),
        ("Sales Performance Report", "Analysis of individual and team sales performance.", "Executive"),
        ("Competitor Analysis", "Detailed analysis of competitors in the market.", "Employee"),
    ]
}

# Function to create and populate each department database with the adjusted schema
def create_department_db(department, data):
    db_name = f"{department.lower()}_db.sqlite"
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    
    # Create the documents table with department and standardized access levels
    c.execute('''
        CREATE TABLE IF NOT EXISTS documents (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            content TEXT,
            department TEXT,
            access_level TEXT,
            created_date TEXT
        )
    ''')
    
    # Populate the table with sample data
    for title, content, access_level in data:
        created_date = datetime.now().strftime("%Y-%m-%d")
        
        # Insert the document with department and access level
        c.execute('''
            INSERT INTO documents (title, content, department, access_level, created_date)
            VALUES (?, ?, ?, ?, ?)
        ''', (title, content, department, access_level, created_date))
    
    conn.commit()
    conn.close()
    print(f"Database for {department} created with standardized access levels and department column.")

# Create each department's database with the new schema and sample data
for dept, documents in departments.items():
    create_department_db(dept, documents)



Database for Finance created with standardized access levels and department column.
Database for HR created with standardized access levels and department column.
Database for Technical created with standardized access levels and department column.
Database for Marketing created with standardized access levels and department column.
Database for Sales created with standardized access levels and department column.


Check the random database created

In [23]:
def check_database(department):
    db_name = f"{department.lower()}_db.sqlite"
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    
    # Fetch all records from the documents table
    c.execute("SELECT * FROM documents")
    records = c.fetchall()
    
    # Print the results
    print(f"\n--- {department} Database ---")
    for record in records:
        print(record)
    
    conn.close()

In [24]:
for dept in departments:
    check_database(dept)


--- Finance Database ---
(1, 'Financial Report Q1', 'This is the content of the Finance Q1 report.', 'Finance', 'Executive', '2024-10-30')
(2, 'Investment Analysis', 'Detailed investment strategies and outcomes.', 'Finance', 'Manager', '2024-10-30')
(3, 'Annual Budget', 'Overview of the annual budget allocations and expenses.', 'Finance', 'Executive', '2024-10-30')
(4, 'Expense Tracking', 'Monthly tracking of departmental expenses.', 'Finance', 'Employee', '2024-10-30')
(5, 'Risk Assessment', 'Assessment of financial risks and mitigation strategies.', 'Finance', 'Employee', '2024-10-30')
(6, 'Quarterly Forecast', 'Forecast of revenue and expenditure for the upcoming quarter.', 'Finance', 'Executive', '2024-10-30')

--- HR Database ---
(1, 'Policy Update', 'New HR policies on leave and remote work.', 'HR', 'Employee', '2024-10-30')
(2, 'Employee Satisfaction Report', 'Analysis of employee satisfaction.', 'HR', 'Manager', '2024-10-30')
(3, 'Onboarding Guide', 'Steps and procedures for n

In [25]:
departments

{'Finance': [('Financial Report Q1',
   'This is the content of the Finance Q1 report.',
   'Executive'),
  ('Investment Analysis',
   'Detailed investment strategies and outcomes.',
   'Manager'),
  ('Annual Budget',
   'Overview of the annual budget allocations and expenses.',
   'Executive'),
  ('Expense Tracking',
   'Monthly tracking of departmental expenses.',
   'Employee'),
  ('Risk Assessment',
   'Assessment of financial risks and mitigation strategies.',
   'Employee'),
  ('Quarterly Forecast',
   'Forecast of revenue and expenditure for the upcoming quarter.',
   'Executive')],
 'HR': [('Policy Update',
   'New HR policies on leave and remote work.',
   'Employee'),
  ('Employee Satisfaction Report',
   'Analysis of employee satisfaction.',
   'Manager'),
  ('Onboarding Guide',
   'Steps and procedures for new employee onboarding.',
   'Employee'),
  ('Performance Review Process',
   'Guidelines for conducting employee performance reviews.',
   'Manager'),
  ('Diversity & I

## Embedding Text into Faiss

In [26]:
import sqlite3
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Initialize the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')
embedding_dim = model.get_sentence_embedding_dimension()

# Initialize the Faiss index
index = faiss.IndexFlatL2(embedding_dim)

# Dictionary to map document IDs to metadata for retrieval
doc_metadata = {}

# Function to embed and add documents to the Faiss index
def embed_documents_from_db(department):
    db_name = f"{department.lower()}_db.sqlite"
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    
    # Fetch all documents from the department's database
    c.execute("SELECT id, title, content, access_level FROM documents")
    documents = c.fetchall()
    
    for doc_id, title, content, access_level in documents:
        # Generate an embedding for the content
        embedding = model.encode(content)
        
        # Add embedding to the Faiss index
        index.add(np.array([embedding], dtype=np.float32))
        
        # Store metadata in a dictionary for reference during retrieval
        doc_metadata[len(doc_metadata)] = {
            "doc_id": doc_id,
            "title": title,
            "department": department,
            "access_level": access_level,
            "content": content
        }
        
        
    
    conn.close()
    print(f"Documents from {department} embedded and added to Faiss index.")

# List of departments to process
departments = ["Finance", "HR", "Technical", "Marketing", "Sales"]

# Process each department and embed documents
for dept in departments:
    embed_documents_from_db(dept)

print("All documents embedded and stored in Faiss index.")


Documents from Finance embedded and added to Faiss index.
Documents from HR embedded and added to Faiss index.
Documents from Technical embedded and added to Faiss index.
Documents from Marketing embedded and added to Faiss index.
Documents from Sales embedded and added to Faiss index.
All documents embedded and stored in Faiss index.


Save Faiss Index and Metadata

In [27]:
import faiss
import json

# Function to save Faiss index and metadata
def save_faiss_index(index, metadata, index_filename="faiss_index.bin", metadata_filename="doc_metadata.json"):
    # Save the Faiss index to a binary file
    faiss.write_index(index, index_filename)
    print(f"Faiss index saved to {index_filename}.")
    
    # Save metadata to a JSON file
    with open(metadata_filename, 'w') as f:
        json.dump(metadata, f)
    print(f"Metadata saved to {metadata_filename}.")

# Save the index and metadata after embedding all documents
save_faiss_index(index, doc_metadata)


Faiss index saved to faiss_index.bin.
Metadata saved to doc_metadata.json.


Load Faiss index and metadata

In [28]:
# import faiss
# import json

# # Function to load Faiss index and metadata
# def load_faiss_index(index_filename="faiss_index.bin", metadata_filename="doc_metadata.json"):
#     # Load the Faiss index from the binary file
#     index = faiss.read_index(index_filename)
#     print(f"Faiss index loaded from {index_filename}.")
    
#     # Load metadata from the JSON file
#     with open(metadata_filename, 'r') as f:
#         metadata = json.load(f)
#     print(f"Metadata loaded from {metadata_filename}.")
    
#     return index, metadata

# # Load the index and metadata for use
# index, doc_metadata = load_faiss_index()


Retrieval with Faiss

In [29]:
def retrieve_similar_documents(query, top_k=5):
    # Encode the query into a vector
    query_embedding = model.encode(query).reshape(1, -1)
    
    # Search for the top_k most similar documents
    distances, indices = index.search(np.array(query_embedding, dtype=np.float32), top_k)
    
    # Display the results
    print("\nTop similar documents:")
    for i, idx in enumerate(indices[0]):
        if idx == -1:
            continue
        doc_info = doc_metadata[idx]
        print(f"{i+1}. Title: {doc_info['title']}, Department: {doc_info['department']}, Access Level: {doc_info['access_level']}")
        print(f"   Content: {doc_info['content'][:100]}...")  # Show a snippet of the content
        print(f"   Distance: {distances[0][i]}\n")

# Example retrieval
retrieve_similar_documents("investment strategies for Q1", top_k=3)


Top similar documents:
1. Title: Investment Analysis, Department: Finance, Access Level: Manager
   Content: Detailed investment strategies and outcomes....
   Distance: 0.6293718814849854

2. Title: Marketing Strategy Q2, Department: Marketing, Access Level: Executive
   Content: Strategies and campaigns planned for Q2....
   Distance: 0.9541587829589844

3. Title: Financial Report Q1, Department: Finance, Access Level: Executive
   Content: This is the content of the Finance Q1 report....
   Distance: 0.9845877289772034



## Routing query with security 

Advantages of routing to separate database embeddings instead a collapsing all database into a single embedding vector and filtering at retrieval level provide additional security and less possibility for corruption and leakage of documents across departments

Define access level logic 

 - Employee: Can only access Employee-level documents in their department.
 - Executive: Can access Employee and Executive documents in any department.
 - Manager: Can access Employee, Executive, and Manager documents in their department; Executive and Employee documents in other departments.
 - Director: Has full access to all documents across all departments.

In [30]:
# Define access control logic
def get_access_levels(user_role, department, doc_department, doc_access_level):
    if user_role == "Employee":
        return doc_department == department and doc_access_level == "Employee"
    elif user_role == "Executive":
        return doc_access_level in ["Employee", "Executive"]
    elif user_role == "Manager":
        if doc_department == department:
            return doc_access_level in ["Employee", "Executive", "Manager"]
        else:
            return doc_access_level in ["Employee", "Executive"]
    elif user_role == "Director":
        return True  # Full access
    return False


In [31]:
import numpy as np

# Function to retrieve documents based on query, user role, and department
def route_query(query, user_role, department=None, top_k=5):
    # Generate embedding for the query
    query_embedding = model.encode(query).reshape(1, -1)
    
    # Search Faiss index for the top_k most similar documents
    distances, indices = index.search(np.array(query_embedding, dtype=np.float32), top_k)
    
    # Filter results based on access level and department
    filtered_results = []
    for i, idx in enumerate(indices[0]):
        if idx == -1:  # No more matches
            continue
        
        # Retrieve document metadata
        doc_info = doc_metadata[idx]  # Access with integer keys
        
        # Check access permissions using the custom access logic
        if get_access_levels(user_role, department, doc_info["department"], doc_info["access_level"]):
            # Add to results if all conditions are met
            filtered_results.append({
                "title": doc_info["title"],
                "content": doc_info["content"],
                "department": doc_info["department"],
                "access_level": doc_info["access_level"],
                "distance": distances[0][i] # similarity measure between the query embedding
            })
    
    return filtered_results



In [32]:
#Employee level results

# Example usage
user_role = "Employee"  # Role of the user querying
department = "Finance"  # Department the user belongs to
query = "investment strategies"  # Query text

# Run the route query and print results
results = route_query(query, user_role, department)
for result in results:
    print(result)

{'title': 'Risk Assessment', 'content': 'Assessment of financial risks and mitigation strategies.', 'department': 'Finance', 'access_level': 'Employee', 'distance': 1.0207815}


In [35]:
#Manager level results

# Example usage
user_role = "Manager"  # Role of the user querying
department = "Technical"  # Department the user belongs to
query = "Budget"  # Query text

# Run the route query and print results
results = route_query(query, user_role, department)
for result in results:
    print(result)

{'title': 'Annual Budget', 'content': 'Overview of the annual budget allocations and expenses.', 'department': 'Finance', 'access_level': 'Executive', 'distance': 0.71796197}
{'title': 'Quarterly Forecast', 'content': 'Forecast of revenue and expenditure for the upcoming quarter.', 'department': 'Finance', 'access_level': 'Executive', 'distance': 1.0298886}
{'title': 'Expense Tracking', 'content': 'Monthly tracking of departmental expenses.', 'department': 'Finance', 'access_level': 'Employee', 'distance': 1.1505497}
{'title': 'Sales Forecast', 'content': 'Projected sales figures for the next quarter.', 'department': 'Sales', 'access_level': 'Executive', 'distance': 1.182723}


## Implementation with Langchain 

LangChain allows you to connect to Faiss and perform vector-based retrieval with minimal setup. 

Define a LangChain Retriever

In [40]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Prepare texts and metadata lists for adding to FAISS
texts = [doc['content'] for doc in doc_metadata.values()]
metadata = [{"title": doc["title"], "department": doc["department"], "access_level": doc["access_level"]} for doc in doc_metadata.values()]

# Initialize and populate the FAISS vector store
vector_store = FAISS.from_texts(texts, embedding_model, metadatas=metadata)

print("FAISS vector store initialized and documents added.")


FAISS vector store initialized and documents added.


 Embed and Add Documents to LangChain Vector Store

In [41]:
# Update your function to use LangChain's FAISS
def embed_documents_from_db_langchain(department):
    db_name = f"{department.lower()}_db.sqlite"
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    
    # Fetch all documents from the department's database
    c.execute("SELECT id, title, content, access_level FROM documents")
    documents = c.fetchall()
    
    # Embed and add each document to LangChain's FAISS
    for doc_id, title, content, access_level in documents:
        # Store metadata alongside content
        metadata = {
            "doc_id": doc_id,
            "title": title,
            "department": department,
            "access_level": access_level
        }
        vector_store.add_texts([content], metadata=[metadata])

    conn.close()
    print(f"Documents from {department} embedded and added to LangChain FAISS index.")

# Process each department and embed documents with LangChain
for dept in departments:
    embed_documents_from_db_langchain(dept)

print("All documents embedded and stored in LangChain's FAISS index.")


Documents from Finance embedded and added to LangChain FAISS index.
Documents from HR embedded and added to LangChain FAISS index.
Documents from Technical embedded and added to LangChain FAISS index.
Documents from Marketing embedded and added to LangChain FAISS index.
Documents from Sales embedded and added to LangChain FAISS index.
All documents embedded and stored in LangChain's FAISS index.


Implement Role-Based Query Routing

In [45]:
def route_query_langchain(query, user_role, department):
    # Define access level restrictions
    access_levels = {
        "Employee": ["Employee"],
        "Manager": ["Employee", "Manager"],
        "Executive": ["Employee", "Manager", "Executive"]
    }
    
    # Perform retrieval
    retriever = vector_store.as_retriever()
    raw_results = retriever.get_relevant_documents(query)

    # Filter results by department and access level
    filtered_results = [
        result for result in raw_results 
        if result.metadata.get('department') == department 
        and result.metadata.get('access_level') in access_levels.get(user_role, [])
    ]
    return filtered_results


In [46]:
results

[{'title': 'Annual Budget',
  'content': 'Overview of the annual budget allocations and expenses.',
  'department': 'Finance',
  'access_level': 'Executive',
  'distance': 0.71796197},
 {'title': 'Quarterly Forecast',
  'content': 'Forecast of revenue and expenditure for the upcoming quarter.',
  'department': 'Finance',
  'access_level': 'Executive',
  'distance': 1.0298886},
 {'title': 'Expense Tracking',
  'content': 'Monthly tracking of departmental expenses.',
  'department': 'Finance',
  'access_level': 'Employee',
  'distance': 1.1505497},
 {'title': 'Sales Forecast',
  'content': 'Projected sales figures for the next quarter.',
  'department': 'Sales',
  'access_level': 'Executive',
  'distance': 1.182723}]

Query and Get Filtered Results

In [47]:
# Example usage
user_role = "Employee"
department = "Finance"
query = "investment strategies"

results = route_query_langchain(query, user_role, department)
for result in results:
    print(result.metadata, result.page_content)

{'title': 'Risk Assessment', 'department': 'Finance', 'access_level': 'Employee'} Assessment of financial risks and mitigation strategies.


In [48]:
user_role = "Manager"  # Role of the user querying
department = "Technical"  # Department the user belongs to
query = "Budget"  # Query text

# Run the route query and print results
results = route_query(query, user_role, department)
for result in results:
    print(result)

{'title': 'Annual Budget', 'content': 'Overview of the annual budget allocations and expenses.', 'department': 'Finance', 'access_level': 'Executive', 'distance': 0.71796197}
{'title': 'Quarterly Forecast', 'content': 'Forecast of revenue and expenditure for the upcoming quarter.', 'department': 'Finance', 'access_level': 'Executive', 'distance': 1.0298886}
{'title': 'Expense Tracking', 'content': 'Monthly tracking of departmental expenses.', 'department': 'Finance', 'access_level': 'Employee', 'distance': 1.1505497}
{'title': 'Sales Forecast', 'content': 'Projected sales figures for the next quarter.', 'department': 'Sales', 'access_level': 'Executive', 'distance': 1.182723}


## Adding RAG - Define a prompt template that combines the retrieved documents with the user’s query.

In [53]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Initialize the model and tokenizer
model_name = "gpt2"  # Replace with any model on Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create a pipeline
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Integrate with LangChain
llm = HuggingFacePipeline(pipeline=hf_pipeline)

def retrieve_documents(query, user_role, department):
    # Define access level restrictions
    access_levels = {
        "Employee": ["Employee"],
        "Manager": ["Employee", "Manager"],
        "Executive": ["Employee", "Manager", "Executive"]
    }
    
    # Perform retrieval
    retriever = vector_store.as_retriever()
    raw_results = retriever.get_relevant_documents(query)

    # Filter results by department and access level
    filtered_results = [
        result for result in raw_results 
        if result.metadata.get('department') == department 
        and result.metadata.get('access_level') in access_levels[user_role]
    ]
    return filtered_results

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/var/folders/zd/pzsd6qy16k3fzg0h42vzr3_40000gn/T/ipykernel_48231/3307074482.py:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [60]:
from langchain.prompts import PromptTemplate

# Define a LangChain prompt template for RAG
prompt_template = PromptTemplate(
    input_variables=["query", "context"],
    template="The document you are looking for:\n\n{context}\n\nbased on your search of: {query} related to"
)

def generate_rag_response(query, user_role, department):
    # Retrieve documents based on query, role, and department
    results = retrieve_documents(query, user_role, department)

    # Combine the content of retrieved documents
    context = " ".join([res.page_content for res in results])

    # Format the prompt with context and query
    prompt = prompt_template.format(query=query, context=context)

    # Generate the response using the LLM
    response = llm(prompt)
    return response

In [61]:
# Example usage
user_role = "Employee"
department = "Finance"
query = "investment strategies"

response = generate_rag_response(query, user_role, department)
print("Generated Response:", response)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Response: The document you are looking for:

Assessment of financial risks and mitigation strategies.

based on your search of: investment strategies related to money markets;

specific risk estimates and assumptions;

accountable for financial exposure;

